#### VAR Automation

#### Description:
###### This file contains the code for VAR automation, along with csv's for intermediate results such as portfolio holdings, price changes for BOTH AAVE AND COMP

###### Author: Nishchal Gaba
###### Created on: January 13, 2022

In [10]:
# Import packages
import os
import time
import sys
import random
from datetime import datetime, timedelta

import json
import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [11]:
import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
from io import StringIO
from datetime import timedelta
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


import matplotlib.pyplot as plt

In [12]:
# Coingecko data fetcher
def getCoinGeckoHistoricalData(_token):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _token(str): Token Symbol (actual token will be fetched from token_names dictionary)
            
        Returns::
            _resDf(pandas dataframe): Returns the dates (yyyy-mm-dd) and prices (USD) in a dataframe
    '''
    try:
        # Making the coingecko historical API request
        coinGeckoTokenCall = token_names[_token]
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coinGeckoTokenCall)+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        
        # Converting the result to JSON
        resJSON = request.json()
        
        # Creating empty dataframe
        resDf = pd.DataFrame(resJSON['prices'], columns = ["timestamp", "price"])
        
        # We divide by 1000 to convert the default milliseconds from CoinGecko to seconds
        # Get the human readable date from timestamp
        resDf['timestamp'] = resDf['timestamp'].apply(lambda x: str(datetime.fromtimestamp(x/1000).date()))
        
        # Return the resultant dataframe
        return resDf
        
        
    except Exception as e:
        print("Exception occured while fetching Coingecko Data")
        return str(e)
    

In [13]:
# AAVE V2 data for a particular date
def aaveV2HistoricalData(_date):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _date(str): date for which data is needed in (%m-%d-%Y) format
            
        Returns::
            _resDf(pandas dataframe): Token, CurrentPrice, availableLiquidity, decimals,
                                        totalLiquidity, utilizationRate, Assets
    '''
    
    try:
        # Starting Index
        ind = 0
        
        # Calling the Aave V2
        aaveV2res = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
        resDf = pd.DataFrame(columns=['Token', 'CurrentPrice', 'availableLiquidity','decimals', 'totalLiquidity','utilizationRate','Assets'])
        
        # Iterating through tokens in AaveV2 API
        for token in aaveV2res:
            try:
                resDf.loc[ind] = [token['symbol']] + [float(token['referenceItem']['priceInUsd'])] + \
                                [float(token['availableLiquidity'])] + [float(token['decimals'])]+ \
                                [float(token['totalLiquidity'])]+ [float(token['referenceItem']['utilizationRate'])] + \
                                [float(token['totalDebt'])]

                ind += 1

            except Exception as e:
                print("**", token, e) #str(asset), str(price_mean), str(price_std), str(cap_mean),str(cap_std), str(vol_mean) , str(vol_std))  
    #             errors.append(token)
                # transactionResponse["error"] = True
                print("Exception occured while fetching Aave V2 Token wise Data")
                return str(e)


        # totalLiquidity == Market
        resDf['totalLiquidity']       = resDf['totalLiquidity'] * resDf['CurrentPrice']
        resDf['Assets']               = resDf['Assets'] * resDf['CurrentPrice']
        resDf['Position']             = resDf['Assets'] - resDf['totalLiquidity']
        resDf['Position($,Billion)']  = resDf['Position'] / pow(10, 9)


        # Return the resultant dataframe
        return resDf
        
    except Exception as e:
        print("Exception occured while fetching Aave V2 API Data")
        return str(e)


In [14]:
# def change():
#     query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

#     V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)
#     V2_client                 = Client(transport=V2_sample_transport)
#     response                  = V2_client.execute(gql(query))

#     # Coingecko Mapping for each token name to fetch historical prices
#     names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         "REN"   : "ren",
#         "RENFIL": "renfil",
#         "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }
    
    
#     columns = []
#     columns.append('timestamp')
#     for key in list(names.keys()):
#         columns.append(key+'_price')
#         columns.append(key+'_%change(10)')


#     df        = pd.DataFrame(columns = columns)
#     LOOK_BACK = 375
#     LIMIT     = 5
#     WBTC      = []

#     request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#     data      = request.json()
#     WBTC      = data['prices']
#     tt        = []
#     P         = []
#     for day in data['prices'][-LOOK_BACK:]:
#         tt.append(day[0]/1000)
#         P.append(day[1])
#     df['WBTC_price'] = P
#     df['timestamp']  = tt

#     track = {}
#     for asset in list(names.keys()):#[:LIMIT]:
#         if asset == 'WBTC':
#             continue
#         request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#         data = request.json()
#         tt = []
#         P = []
        
#         if len(data['prices']) < LOOK_BACK:
#             req = LOOK_BACK - len(data['prices'])
#             addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
#             data['prices'] = addition + data['prices']
#             track[asset] = req
#         else:
#             data['prices'] = data['prices'][-LOOK_BACK:]
        
#         for day in data['prices']:
#             tt.append(day[0]/1000)
#             P.append(day[1])
#         df[asset+'_price'] = P
#         df['timestamp']  = tt

#     df['timestamp'] = tt

#     for asset in list(names.keys()):#[:LIMIT]:
        
#         diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(10)'] = change
        
#         diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(1)'] = change
        
#     for key in track.keys():
#         df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

#     df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
#     df  = df.iloc[::-1]
#     df  = df.reset_index()
#     df = df.drop(['index'], axis=1)

        
#     return df

In [15]:
# Coingecko Mapping for each token name to fetch historical prices
names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        "REN"   : "ren",
        "RENFIL": "renfil",
        "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }

In [16]:
# Funtion to give historical price changes for each token mentioned in Names Dictionary
def change():
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        P    = []
        if request.status_code != 200:
            data = {}
            data['prices'] = WBTC[ -LOOK_BACK: ]
            for day in data['prices']:
                P.append(day)
        else:   
            data = request.json()
            tt   = []
            

            if len(data['prices']) < LOOK_BACK:
                req = LOOK_BACK - len(data['prices'])
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                track[asset] = req
            else:
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                tt.append(day[0]/1000)
                P.append(day[1])
                
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    
    print("PRICES FETECHED")
    
    unknown_assets = ['PAX']
    for asset in list(names.keys()):#[:LIMIT]:
        try:
            change = []
            diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(10)'] = change

            diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(1)'] = change  
            
        except Exception as e:
            unknown_assets.append(asset)
            print("Error ", asset, e)
    
    for asset in unknown_assets:
        df[asset+'_%change(1)']  = df['WBTC_%change(1)'] 
        df[asset+'_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(1)'] = df['WBTC_%change(1)'] 
        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df
change_df = change()

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'
CHECK POINT 


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [18]:
# query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

In [19]:
# V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)

In [20]:
# Required Date
# By default set to current date
# requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate = '01-05-2022' #"01-10-2022" #mm-dd-yyyy

In [21]:
def VARDate(_date):
    '''
        Gets the VAR results, AaveV2 for a particular position and price change dataframe
        
        Args::
            _date(str): Date in "%m-%d-%Y" format
        
        Returns::
            aaveV2Res(pandas dataframe): AaveV2 Historical Data for the given date
            df (pandas dataframe): Price change dataframe
    '''
    # Aave V2 Data for particular date
    aaveV2Res = aaveV2HistoricalData(_date)
    
    # Price Change Dataframe
    df = change()
    
    # Flipping the date format
    reqDate = datetime.strptime(_date, '%m-%d-%Y').strftime('%Y-%m-%d')
    
    # Converting str to datetime
    df['date']= pd.to_datetime(df['date'])
    # Remove dates above the required date
    mask = (df['date'] <= reqDate)
    df = df.loc[mask]
    
    # Converting the date column back to str
    df['date']=df['date'].astype(str)
    
    REQUIRED_1 = []
    REQUIRED_10 = []
    historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
    hist_ind = 0
    for required_row in df.iterrows():
        required_row = required_row[1]
        SUM_10 = []
        SUM_1 = []
            
        for row in aaveV2Res.iterrows():
            try:
                SUM_10.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(10)'])/100))
                SUM_1.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(1)'])/100))
            except Exception as e:
                print("ERROR", e)
                pass

        REQUIRED_1.append(sum(SUM_1))
        REQUIRED_10.append(sum(SUM_10))

        historicalVAR.loc[hist_ind] = [required_row['date']] + [sum(SUM_10)] + [sum(SUM_1)]
        hist_ind += 1 
        
    # Historical VAR for past 362 days    
    historicalVAR = historicalVAR[:365]
    
    
    
    REQUIRED_10 = historicalVAR.sort_values('VAR_10').iloc[3]
    REQUIRED_1 = historicalVAR.sort_values('VAR_1').iloc[3]

    REQUIRED_10_95 = historicalVAR.sort_values('VAR_10').iloc[9]
    REQUIRED_1_95 = historicalVAR.sort_values('VAR_1').iloc[9]

    var_date_10_99p = REQUIRED_10['date']
    var_date_10_95p = REQUIRED_10_95['date']
    var_date_1_99p = REQUIRED_1['date']
    var_date_1_95p = REQUIRED_1_95['date']
    
#     print(historicalVAR)
    
    testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
    ind = 0
    
    AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
    
    for i in AAVE_API_RESULT:
        testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                    + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
        ind += 1

    TOTAL_LIABILITIES = aaveV2Res['totalLiquidity'].sum()
    TOTAL_ASSETS      = aaveV2Res['Assets'].sum()
    VAR_10_DAY_99_P = str(REQUIRED_10['VAR_10'])
    VAR_DATE_10_DAY_99_P = var_date_10_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['10_day_95p'] = str(REQUIRED_10_95['VAR_10'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_10_day_95p'] = var_date_10_95p

#     # transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1[5])
#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_99p'] = var_date_1_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_95p'] = str(REQUIRED_1_95['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_95p'] = var_date_1_95p

    RELATIVE_VAR_ASSETS = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_ASSETS)
    RELATIVE_VAR_LIABILITIES = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_LIABILITIES)
    #TOTAL_ASSETS
    #TOTAL_LIABILITIES
    return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [22]:
# aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)
# Date in %m-%d-%Y format
aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CHECK POINT 


In [23]:
# Checking aaveV2Res
aaveV2Res
# To be taken [Token, Position, Position($,Billion)]
# REMEBER TO PUT PLATFORM

,Token,CurrentPrice,availableLiquidity,decimals,totalLiquidity,utilizationRate,Assets,Position,"Position($,Billion)"
0,TUSD,1.006012,4.909780e+07,18.0,1.168372e+08,0.628140,6.744422e+07,-4.939296e+07,-0.049393
1,RAI,3.000366,2.068256e+06,18.0,3.116527e+07,0.705354,2.495975e+07,-6.205524e+06,-0.006206
2,GUSD,1.000000,2.911591e+06,2.0,1.442335e+07,0.790259,1.151176e+07,-2.911591e+06,-0.002912
3,YFI,34209.643683,1.469237e+03,18.0,5.384758e+07,0.057023,3.585490e+06,-5.026209e+07,-0.050262
4,BAT,1.342874,4.199938e+06,18.0,6.179377e+06,0.087247,5.393911e+05,-5.639986e+06,-0.005640
5,MANA,2.847777,1.337645e+07,18.0,3.930868e+07,0.024051,1.215537e+06,-3.809314e+07,-0.038093
6,DPI,264.561221,1.789095e+05,18.0,4.759038e+07,0.005390,2.578583e+05,-4.733252e+07,-0.047333
7,UNI,17.820205,2.650525e+06,18.0,4.772944e+07,0.008988,4.965366e+05,-4.723290e+07,-0.047233
8,WBTC,43266.134149,2.647995e+04,8.0,1.172187e+09,0.022934,2.650203e+07,-1.145685e+09,-1.145685
9,REN,0.517895,4.418143e+07,18.0,2.343989e+07,0.021415,5.585670e+05,-2.288133e+07,-0.022881


In [24]:
# Change in the price data for those particular dates

In [25]:
VAR_10_DAY_99_P

'-4.143725804542771'

In [26]:
VAR_DATE_10_DAY_99_P

'2021-01-06'

In [27]:
type(priceChangeDf['date'])

pandas.core.series.Series

In [28]:
# Token change in price for AAVE VAR Date
idx = priceChangeDf.index[priceChangeDf['date'] == VAR_DATE_10_DAY_99_P].tolist()
priceChangeVARDATE = priceChangeDf.loc[idx]
# _index = priceChangeDf.index[mask]
# priceChangeDf[_index]

In [29]:
priceChangeVARDATE["TUSD"+"_price"]

373    1.003462
Name: TUSD_price, dtype: float64

In [30]:
priceChangeVARDATE["TUSD"+"_%change(10)"]

373    0.170563
Name: TUSD_%change(10), dtype: object

In [31]:
# Contribution to VAR
# -1 * (TOKEN_PRICE * TOKEN_POSITION * TOKEN%CHANGE)

In [32]:
# PLATFORM, TOKEN, POSITION_ON_INPUT_DATE, POSITION_in_B_ON_INPUT_DATE, TOKEN_%_CHANGE_ON_VAR_DATE, CONTRIBTION TO VAR

In [33]:
column_names = ["PLATFORM", "TOKEN", "POSITION_ON_INPUT_DATE", "POSITION_ON_INPUT_DATE_BILLIONS",
               "TOKEN_%_CHANGE_ON_VAR_DATE", "CONTRIBUTION_TO_VAR"]
resultDf = pd.DataFrame(columns = column_names)

In [34]:
# Itearing through the position in Aave

# for token in aaveV2Res["Token"]:
for idx, row in aaveV2Res.iterrows():
    try:
        temp = []
        
        temp.append("AAVE")
        temp.append(str(aaveV2Res["Token"][idx]))
        temp.append(aaveV2Res["Position"][idx])
        temp.append(aaveV2Res["Position($,Billion)"][idx])
        temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))

# print(temp)

In [35]:
# Sample Final Res
resultDf

,PLATFORM,TOKEN,POSITION_ON_INPUT_DATE,POSITION_ON_INPUT_DATE_BILLIONS,TOKEN_%_CHANGE_ON_VAR_DATE,CONTRIBUTION_TO_VAR
0,AAVE,TUSD,-4.939296e+07,-0.049393,0.170563,0.008425
1,AAVE,RAI,-6.205524e+06,-0.006206,28.852430,0.179044
2,AAVE,GUSD,-2.911591e+06,-0.002912,0.293206,0.000854
3,AAVE,YFI,-5.026209e+07,-0.050262,5.766264,0.289824
4,AAVE,BAT,-5.639986e+06,-0.005640,8.584189,0.048415
5,AAVE,MANA,-3.809314e+07,-0.038093,3.946910,0.150350
6,AAVE,DPI,-4.733252e+07,-0.047333,53.720534,2.542728
7,AAVE,UNI,-4.723290e+07,-0.047233,85.284473,4.028233
8,AAVE,WBTC,-1.145685e+09,-1.145685,28.852430,33.055800
9,AAVE,REN,-2.288133e+07,-0.022881,28.852430,0.660182


### Compound Code

In [36]:
def change():

    names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        # "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        # "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        # "REN"   : "ren",
        "RENFIL": "renfil",
        # "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]
        
        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:
        
        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change
        
        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df

In [37]:
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi",  
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

def change():    
    columns = []
    columns.append('timestamp')
    print(names)
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        print("DEBUG 1 ", asset)
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        # print("DEBUG 1 ", data)
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]

        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
#         df['timestamp']  = tt
#     df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:

        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change

        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df[:-10]
    print("CHECK POINT ")
    return df[1:]

change_df = change()

# min_var = 9999999

{'DAI': 'dai', 'TUSD': 'true-usd', 'USDC': 'usd-coin', 'USDT': 'tether', 'ETH': 'ethereum', 'LINK': 'chainlink', 'MKR': 'maker', 'UNI': 'uniswap', 'WBTC': 'wrapped-bitcoin', 'WBTC2': 'wrapped-bitcoin', 'SUSHI': 'sushi', 'YFI': 'yearn-finance', 'AAVE': 'aave', 'BAT': 'basic-attention-token', 'ZRX': '0x', 'REP': 'augur', 'SAI': 'sai'}
DEBUG 1  DAI
DEBUG 1  TUSD
DEBUG 1  USDC
DEBUG 1  USDT
DEBUG 1  ETH
DEBUG 1  LINK
DEBUG 1  MKR
DEBUG 1  UNI
DEBUG 1  WBTC2
DEBUG 1  SUSHI
DEBUG 1  YFI
DEBUG 1  AAVE
DEBUG 1  BAT
DEBUG 1  ZRX
DEBUG 1  REP
DEBUG 1  SAI
CHECK POINT 


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [38]:
change_df.head()

,timestamp,DAI_price,DAI_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDT_price,USDT_%change(10),ETH_price,...,WBTC_%change(1),WBTC2_%change(1),SUSHI_%change(1),YFI_%change(1),AAVE_%change(1),BAT_%change(1),ZRX_%change(1),REP_%change(1),SAI_%change(1),date
1,1.642032e+09,1.001430,-0.0298551,1.001834,0.10263,1.001409,0.108038,1.000972,0.0488767,3378.035895,...,2.41734,2.41734,1.72256,2.63012,1.98119,4.5334,3.92047,3.23535,0,2022-01-13
2,1.641946e+09,0.999613,-0.494773,1.000153,-0.557577,0.999859,-0.508185,1.000418,-0.517607,3246.413723,...,2.03631,2.03631,4.23861,1.99777,5.13704,4.90393,3.49427,5.37757,0,2022-01-12
3,1.641859e+09,1.001071,0.170157,1.002476,0.424748,1.000715,0.0284644,1.001096,0.251778,3086.729002,...,-0.181559,-0.181559,-3.77384,-1.97316,-3.98116,-2.77533,-4.79252,-4.2775,0,2022-01-11
4,1.641773e+09,1.002857,-0.059015,1.000299,-0.169955,1.000593,-0.263729,0.999476,-0.185822,3156.967262,...,0.445028,0.445028,3.44492,-1.71643,-1.17769,0.412506,1.95901,0.724551,0,2022-01-10
5,1.641686e+09,0.997979,-0.511445,0.997312,-0.101253,0.999403,-0.581565,0.997693,-0.457192,3101.040902,...,-0.653672,-0.653672,-6.24161,-8.69001,0.953256,-4.65892,-5.25208,-4.46708,0,2022-01-09


In [39]:
# !mkdir AAVE_COMPOUND_DATA
!mkdir COMPOUND_DATA

mkdir: cannot create directory ‘COMPOUND_DATA’: File exists


In [40]:
# names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         # "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         # "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         # # "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         # "REN"   : "ren",
#         "RENFIL": "renfil",
#         # "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }

In [41]:
# requiredDate

In [42]:
# Compound Tokens
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi", # check -- - - - - -- 
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

In [43]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
for ctoken in res.json()['cToken'][3:]:
    
    print(ctoken['symbol'])
    if ctoken['symbol'] == 'cCOMP':
        continue

    ind += 1
    for key in ctoken.keys():
        try:
            df.at[ind,key ] = ctoken[key]['value']
        except:
            df.at[ind,key ] = ctoken[key]

    

    address   = ctoken['token_address']

    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    prices_df = pd.DataFrame(columns=['timestamp', 'price'])
    timestamp = []
    prices    = []

    for req in request.json()['prices']:
        timestamp.append(int(float(req[0])/1000))
        prices.append(req[1])

    prices_df['timestamp'] = timestamp
    prices_df['price']     = prices

    date= []
    for time in timestamp:
        date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
    prices_df['date']  = date

cWBTC
cMKR
cZRX
cYFI
cSUSHI
cLINK
cBAT
cTUSD
cCOMP
cAAVE
cUNI
cWBTC2
cDAI
cETH
cUSDT
cUSDC


In [44]:
prices_df

,timestamp,price,date
0,1538697600,1.006242,2018-10-05
1,1538784000,1.001530,2018-10-06
2,1538870400,1.001177,2018-10-07
3,1538956800,1.001906,2018-10-08
4,1539043200,1.001983,2018-10-09
...,...,...,...
1193,1641772800,1.000593,2022-01-10
1194,1641859200,1.000715,2022-01-11
1195,1641945600,0.999859,2022-01-12
1196,1642032000,1.001409,2022-01-13


In [45]:
requiredDate
datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')

'2022-01-05'

In [46]:
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
list(res.json()['cToken'][0])

['borrow_cap',
 'borrow_rate',
 'cash',
 'collateral_factor',
 'comp_borrow_apy',
 'comp_supply_apy',
 'exchange_rate',
 'interest_rate_model_address',
 'name',
 'number_of_borrowers',
 'number_of_suppliers',
 'reserve_factor',
 'reserves',
 'supply_rate',
 'symbol',
 'token_address',
 'total_borrows',
 'total_supply',
 'underlying_address',
 'underlying_name',
 'underlying_price',
 'underlying_symbol']

In [47]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
# MASTER_DATE = '2021-12-20'
MASTER_DATE = datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')
for ctoken in res.json()['cToken']:
    try:
        ind += 1
        for key in ctoken.keys():
            try:
                df.at[ind,key ] = ctoken[key]['value']
            except:
                df.at[ind,key ] = ctoken[key]


        address   = ctoken['token_address']

        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        prices_df = pd.DataFrame(columns=['timestamp', 'price'])
        timestamp = []
        prices    = []

        for req in request.json()['prices']:
            timestamp.append(int(float(req[0])/1000))
            prices.append(req[1])

        prices_df['timestamp'] = timestamp
        prices_df['price']     = prices

        date= []
        for time in timestamp:
            date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
        prices_df['date']  = date

#         datestamp = '2021-12-14'
        datestamp = MASTER_DATE
        max_timstamp = int(datetime.strptime(datestamp, '%Y-%m-%d').timestamp()) + 86400
        SUPPLY_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        BORROW_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        DF = pd.DataFrame()
        BIND = 0
        SIND = 0
        for _ in range(5):
            min_timestamp = max_timstamp  -  30 * 86400 
            res = requests.get("https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=50" % ( address,min_timestamp, max_timstamp));    
            max_timstamp = max_timstamp   - 30 * 86400 
            temp = res.json()


            for tem in temp['total_borrows_history']:
                BORROW_DF.loc[BIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]                     
                BIND += 1


            for tem in temp['total_supply_history']:
                SUPPLY_DF.loc[SIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]
                SIND += 1                      


            print("***")

        BORROW_DF = BORROW_DF.sort_values(['block_timestamp'])
        BORROW_DF['DATE'] = BORROW_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        BORROW_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)
        SUPPLY_DF = SUPPLY_DF.sort_values(['block_timestamp'])
        SUPPLY_DF['DATE'] = SUPPLY_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        SUPPLY_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)

        SUPPLY_DF['supply_tokens'] = pd.to_numeric(SUPPLY_DF['value']) *  float(temp['exchange_rates'][-1]['rate'])


        BORROW_DF = BORROW_DF[::-1]
        SUPPLY_DF = SUPPLY_DF[::-1]
        DF['timestamp']    = BORROW_DF['block_timestamp']
        DF['BORROW']       = BORROW_DF['value']
        DF['SUPPLY']       = SUPPLY_DF['supply_tokens']
        DF['DATE']         = BORROW_DF['DATE']
        DF['SUPPLY_VALUE'] = SUPPLY_DF['value']


        assigned_price = []
        for indW in DF['DATE'] :
            a = prices_df[prices_df['date']==str(indW)]
            assigned_price.append(a['price'].values[0])

        DF['prices'] = assigned_price

        DF['total_supply'] =  DF['SUPPLY'] * DF['prices']
        DF['total_borrow'] =  pd.to_numeric(DF['BORROW']) * pd.to_numeric(DF['prices'])
        DF['POSITION']     =  DF['total_borrow'] - DF['total_supply']
        DF['POSITION($)']  =  DF['POSITION'] / pow(10,9)
        DF.to_csv('COMPOUND_DATA/DF_'+ctoken['symbol'][1:] + '.csv')



        print("**********************", DF.shape)
    except Exception as e:
        print("ERROR ", e, ctoken['symbol'])
    

***
***
***
***
***
********************** (59, 10)
***
***
***
***
***
ERROR  index 0 is out of bounds for axis 0 with size 0 cSAI
ERROR  'USDP' cUSDP
***
***
***
***
***
********************** (135, 10)
***
***
***
***
***
********************** (137, 10)
***
***
***
***
***
********************** (149, 10)
***
***
***
***
***
********************** (141, 10)
***
***
***
***
***
********************** (147, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (148, 10)
***
***
***
***
***
********************** (145, 10)
ERROR  'COMP' cCOMP
***
***
***
***
***
********************** (141, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)


In [48]:
fils = os.listdir('COMPOUND_DATA/')

In [49]:
cols = []
for fil in fils:
    try:
        aa = fil.split('.')[0].split('_')
    #     print(aa[1])
        cols.append(aa[1])
    except:
        pass

In [50]:
FINAL ={}
for key in cols:
    try:
        df = pd.read_csv('COMPOUND_DATA/'+'DF_'+key+'.csv')
        df = df.sort_values(['timestamp'])
#         print(df.head())
        df = df[::-1]
        FINAL[key] = {'Assets': df['POSITION($)'],
                      'date':df['DATE'],
                      'MARKET':df['total_supply'],
                      'BORROW':df['total_borrow']
                     }
    
    except Exception as e:
        print(e, key)


In [51]:
ff = []
for f in FINAL['ETH']['date']:
    ff.append(f)

In [52]:
FINAL["AAVE"]["date"]

0      2022-01-02
1      2022-01-01
2      2021-12-31
3      2021-12-30
4      2021-12-29
          ...    
136    2021-08-13
137    2021-08-12
138    2021-08-11
139    2021-08-10
140    2021-08-09
Name: DATE, Length: 141, dtype: object

In [53]:
FINAL_DF = pd.DataFrame(columns= cols)
ind = 0
for i in range(100):
    assets = []
    for col in cols:
        try:
            assets.append(FINAL[col]['Assets'][i])
        except:
            assets.append([])
    
    FINAL_DF.loc[ind] = assets
    ind += 1

/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [54]:
FINAL_DF['DATE'] = ff[:100]
FINAL_DF['timestamp'] = FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
FINAL_DF = FINAL_DF.sort_values('timestamp')
FINAL_DF = FINAL_DF[::-1]

In [55]:
# Assets for Compound Datewise
FINAL_DF

,AAVE,BAT,DAI,ETH,LINK,MKR,REP,SUSHI,TUSD,UNI,USDC,USDT,WBTC,WBTC2,YFI,ZRX,DATE,timestamp
0,-0.002608,-0.124891,-0.757592,-5.087054,-0.051496,-0.003985,-1.00163e-05,-0.001016,-0.037218,-0.090881,-0.617799,-0.172820,-0.110538,-1.423870,-0.001370,-0.101979,2022-01-05,1.641341e+09
1,-0.003150,-0.125242,-0.749345,-5.085617,-0.051336,-0.004215,8.86386e-06,-0.001073,-0.037204,-0.091160,-0.669254,-0.174319,-0.112602,-1.443595,-0.001443,-0.107356,2022-01-04,1.641254e+09
2,-0.003254,-0.120475,-0.757688,-5.180065,-0.047535,-0.004494,8.25518e-06,-0.001098,-0.037181,-0.090251,-0.666049,-0.168984,-0.111083,-1.467283,-0.001403,-0.101263,2022-01-03,1.641168e+09
3,-0.002921,-0.116819,-0.761214,-5.127199,-0.044738,-0.004510,8.01429e-06,-0.001237,-0.037365,-0.086399,-0.653033,-0.173920,-0.113723,-1.473303,-0.001283,-0.098426,2022-01-02,1.641082e+09
4,-0.003136,-0.115126,-0.748610,-5.127352,-0.042882,-0.004317,7.97565e-06,-0.001343,-0.037181,-0.083823,-0.653351,-0.165872,-0.121189,-1.470908,-0.001175,-0.091709,2022-01-01,1.640995e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.009366,-0.069531,-1.081011,-5.980912,-0.163784,-0.000809,[],-0.000294,-0.016008,-0.207550,-0.869690,-0.178716,-0.129819,-1.956963,-0.001128,-0.105084,2021-10-02,1.633133e+09
96,-0.009648,-0.065451,-1.078420,-5.462986,-0.148959,-0.001790,[],-0.000353,-0.015505,-0.189261,-0.854045,-0.175191,-0.130726,-1.776202,-0.001105,-0.095954,2021-10-01,1.633046e+09
97,-0.008561,-0.064060,-1.017522,-5.190630,-0.142061,-0.002257,[],-0.000421,-0.015558,-0.187949,-0.868271,-0.213280,-0.128945,-1.689086,-0.001172,-0.089328,2021-09-30,1.632960e+09
98,-0.009293,-0.067180,-1.008638,-5.095162,-0.137008,-0.002569,[],-0.000464,-0.017348,-0.187311,-0.860990,-0.240410,-0.123461,-1.663105,-0.001148,-0.087484,2021-09-29,1.632874e+09


In [56]:
change_df

,timestamp,DAI_price,DAI_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDT_price,USDT_%change(10),ETH_price,...,WBTC_%change(1),WBTC2_%change(1),SUSHI_%change(1),YFI_%change(1),AAVE_%change(1),BAT_%change(1),ZRX_%change(1),REP_%change(1),SAI_%change(1),date
1,1.642032e+09,1.001430,-0.0298551,1.001834,0.10263,1.001409,0.108038,1.000972,0.0488767,3378.035895,...,2.41734,2.41734,1.72256,2.63012,1.98119,4.5334,3.92047,3.23535,0,2022-01-13
2,1.641946e+09,0.999613,-0.494773,1.000153,-0.557577,0.999859,-0.508185,1.000418,-0.517607,3246.413723,...,2.03631,2.03631,4.23861,1.99777,5.13704,4.90393,3.49427,5.37757,0,2022-01-12
3,1.641859e+09,1.001071,0.170157,1.002476,0.424748,1.000715,0.0284644,1.001096,0.251778,3086.729002,...,-0.181559,-0.181559,-3.77384,-1.97316,-3.98116,-2.77533,-4.79252,-4.2775,0,2022-01-11
4,1.641773e+09,1.002857,-0.059015,1.000299,-0.169955,1.000593,-0.263729,0.999476,-0.185822,3156.967262,...,0.445028,0.445028,3.44492,-1.71643,-1.17769,0.412506,1.95901,0.724551,0,2022-01-10
5,1.641686e+09,0.997979,-0.511445,0.997312,-0.101253,0.999403,-0.581565,0.997693,-0.457192,3101.040902,...,-0.653672,-0.653672,-6.24161,-8.69001,0.953256,-4.65892,-5.25208,-4.46708,0,2022-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,1.578614e+09,1.002732,2.12226,1.000020,-0.257931,1.000419,-0.0833051,0.999479,0.120977,137.860559,...,-3.10762,-3.10762,-3.10762,-3.10762,-3.10762,1.16746,-2.43441,-2.62453,0.199728,2020-01-10
736,1.578528e+09,1.000406,1.61287,0.996705,-0.35493,0.996342,-0.389219,0.996537,-0.319909,140.273935,...,-0.667996,-0.667996,-0.667996,-0.667996,-0.667996,-5.40615,12.0742,-0.331698,0.113107,2020-01-09
737,1.578442e+09,0.995636,2.42278,0.999350,-0.317283,0.999140,-0.249331,0.997681,-0.16141,143.019432,...,4.64645,4.64645,4.64645,4.64645,4.64645,0.925568,-3.80894,-1.29527,-0.450304,2020-01-08
738,1.578355e+09,0.998680,2.48368,0.998690,-0.22173,0.999543,-0.198063,0.998183,-0.167694,143.806398,...,5.94334,5.94334,5.94334,5.94334,5.94334,2.30919,4.73891,3.83729,0.142036,2020-01-07


In [57]:
VAR = []
VAR_DATE = []
for ind, ROW in enumerate(FINAL_DF.iterrows()):
    REQUIRED_10 = []
    for required_row in change_df[ind:ind+365].iterrows():
        SUM_10 = []
        for key in ROW[1].keys()[:-3]:
            if key == 'WBTC2':
                key = 'WBTC'
            if isinstance(ROW[1][key], list):
                ROW[1][key] = 0
            SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
        REQUIRED_10.append(sum(SUM_10))
    VAR.append(sorted(REQUIRED_10, key=float)[3])
    VAR_DATE.append(change_df[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])

In [58]:
BORROW_DF

,block_number,block_timestamp,value,DATE
49,13942167,1641355569,2276084637.47815888,2022-01-05
47,13934283,1641251889,2257476388.59752436,2022-01-04
45,13926398,1641148208,2226270593.84307691,2022-01-03
44,13922456,1641096368,2152889758.95632855,2022-01-02
42,13914572,1640992688,2211020428.58465392,2022-01-01
...,...,...,...,...
207,12988153,1628810279,3230578035.45465452,2021-08-13
206,12984211,1628758439,3217720622.24404443,2021-08-12
204,12976326,1628654759,3144302713.29184582,2021-08-11
202,12968442,1628551079,3087900354.74393670,2021-08-10


In [59]:
# VAR_DATE

In [60]:
FINAL_DF["DATE"]

0     2022-01-05
1     2022-01-04
2     2022-01-03
3     2022-01-02
4     2022-01-01
         ...    
95    2021-10-02
96    2021-10-01
97    2021-09-30
98    2021-09-29
99    2021-09-28
Name: DATE, Length: 100, dtype: object

In [61]:
# Sample Final Res Columns
resultDf.columns

Index(['PLATFORM', 'TOKEN', 'POSITION_ON_INPUT_DATE',
       'POSITION_ON_INPUT_DATE_BILLIONS', 'TOKEN_%_CHANGE_ON_VAR_DATE',
       'CONTRIBUTION_TO_VAR'],
      dtype='object')

In [62]:
# resultDf

In [63]:
FINAL_DF.columns[:-2][0]

'AAVE'

In [64]:
requiredDate

'01-05-2022'

In [65]:
FINAL_DF["AAVE"].loc[0]

-0.002607747383590364

In [66]:
VAR_DATE[0]

'2021-05-11'

In [67]:
idx = change_df.index[change_df['date'] == str(VAR_DATE[0])]
priceChangeVARDATE = change_df.loc[idx]

In [68]:
priceChangeVARDATE

,timestamp,DAI_price,DAI_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDT_price,USDT_%change(10),ETH_price,...,WBTC_%change(1),WBTC2_%change(1),SUSHI_%change(1),YFI_%change(1),AAVE_%change(1),BAT_%change(1),ZRX_%change(1),REP_%change(1),SAI_%change(1),date
248,1.620691e+09,1.003394,0.400227,1.001942,-0.0250118,0.999028,-0.159781,1.007187,0.58062,3979.608652,...,-3.97152,-3.97152,-11.7483,16.8803,-5.47237,-7.90729,-11.2555,-14.9401,4.52943,2021-05-11


In [70]:
idx = change_df.index[change_df['date'] == str(VAR_DATE[0])]
priceChangeVARDATE = change_df.loc[idx]
    
for token in list(FINAL_DF.columns[:-2]):
    try:
#         print(token)
        temp = []
        
        temp.append("COMPOUND")
        temp.append(str(token))
        temp.append(float(FINAL_DF[token].loc[0]*pow(10,9)))
        temp.append(float(FINAL_DF[token].loc[0]))
        temp.append(priceChangeVARDATE[token+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(FINAL_DF[token].loc[0]) * float(priceChangeVARDATE[token+"_%change(10)"].iloc[0])))
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))
# print(temp)

In [71]:
resultDf

,PLATFORM,TOKEN,POSITION_ON_INPUT_DATE,POSITION_ON_INPUT_DATE_BILLIONS,TOKEN_%_CHANGE_ON_VAR_DATE,CONTRIBUTION_TO_VAR
0,AAVE,TUSD,-4.939296e+07,-0.049393,0.170563,0.008425
1,AAVE,RAI,-6.205524e+06,-0.006206,28.852430,0.179044
2,AAVE,GUSD,-2.911591e+06,-0.002912,0.293206,0.000854
3,AAVE,YFI,-5.026209e+07,-0.050262,5.766264,0.289824
4,AAVE,BAT,-5.639986e+06,-0.005640,8.584189,0.048415
5,AAVE,MANA,-3.809314e+07,-0.038093,3.946910,0.150350
6,AAVE,DPI,-4.733252e+07,-0.047333,53.720534,2.542728
7,AAVE,UNI,-4.723290e+07,-0.047233,85.284473,4.028233
8,AAVE,WBTC,-1.145685e+09,-1.145685,28.852430,33.055800
9,AAVE,REN,-2.288133e+07,-0.022881,28.852430,0.660182


In [72]:
resultDf.to_csv(str(requiredDate)+"_AAVE_COMPOUND_VAR.csv", index=False)

In [ ]:

# # for token in aaveV2Res["Token"]:
# for idx, row in aaveV2Res.iterrows():
#     try:
#         temp = []
        
#         temp.append("AAVE")
#         temp.append(str(aaveV2Res["Token"][idx]))
#         temp.append(aaveV2Res["Position"][idx])
#         temp.append(aaveV2Res["Position($,Billion)"][idx])
#         temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
#         temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
#         # Add the values to the latest row
#         df_length = len(resultDf)
#         resultDf.loc[df_length] = temp
        
#     except:
#         print("Exception occured for the token: "+str(token))

# # print(temp)

In [ ]:
# idx = FINAL_DF.index[FINAL_DF['DATE'] == MASTER_DATE].tolist
# print(idx)
# # COMPpriceChangeVARDATE = FINAL_DF.loc[idx]

In [ ]:
# FINAL_DF.index[FINAL_DF['DATE'] == MASTER_DATE]

In [ ]:
# FINAL_DF["DATE"]

In [ ]:
# MASTER_DATE